In [1]:
%bookmark HOME "/home/jupyter/meta_XLM/XLM" 
%cd -b HOME

(bookmark:HOME) -> /home/jupyter/meta_XLM/XLM
/home/jupyter/meta_XLM/XLM


# DATA

In [3]:
import os
import csv

In [5]:
langues_all = [
            # langues presentes dans l'ancien et le nouveau testament
           'Francais', "Anglais",  "BIBALDA_TA_PELDETTA", 'Bulu',  'Guiziga', "Fulfulde_Adamaoua",  
           "Fulfulde_DC", 'KALATA_KO_SC_Gbaya', 'KALATA_KO_DC_Gbaya', 'Kapsiki_DC', 'Tupurri',
           
           # langues presentes uniquement dans le nouveau testament
          'Bafia', 'Dii', 'Ejagham', 'Ghomala', 'Vute', 'Limbum', 'MKPAMAN_AMVOE_Ewondo', 'Mofa', 
           "Mofu_Gudur", "Ngiemboon", 'Doyayo', "Guidar", 'Peere_Nt&Psalms', 'Samba_Leko', 
           "Du_na_sdik_na_wiini_Alaw"
]

In [6]:
def get_data_from_bible(csv_path, livres, output_dir, data_type = "para", langues=[], cell_error = "__Error__"):
    """
    csv_path : dossier contenant le dossier csvs
    livres : liste des livres de la bibles à considerer (il doit existe pour chacun de ces livres un fichier livre.csv dans ../csvs/)
    output : dossier dans lequel sera crée les dossier d'objectif (mono ou para)
    data_type : monolingual one or parallel one
    langues : liste des langues à considérer (ces langues doivent figurer dans la liste des langues presentes plus haut)
    cell_error : texte qui à servir de marquage des paires de texte érronées lors du webscrapping (ces paires sont exclues des données)
    """

    # Si aucune langue n'est spécifiée, on selectionne tout les langues
    if langues == [] :
        langues = langues_all
    
    l = len(langues)
    for i in range(l-1):
        for j in range(i+1, l):
            li = langues[i]
            lj = langues[j]
            print(li + '-' + lj)

            repertoire = output_dir+"/"+data_type
            if not os.path.exists(repertoire):
                os.makedirs(repertoire)

            if data_type == "para":
                repertoire = repertoire +"/"+ li +'-'+ lj
                if not os.path.exists(repertoire):
                    os.makedirs(repertoire)
                repertoire = [repertoire + "/" + li + '-' + lj + "." for _ in range(2)]
            elif data_type == "mono":
                repertoire = [repertoire + "/" + li, repertoire +"/"+ lj]
                for rep in repertoire :
                    if not os.path.exists(rep):
                        os.makedirs(rep)
                repertoire = [r +"/" for r in repertoire]

            with open(repertoire[0] + li + '.txt', 'w') as txtfile1:
                with open(repertoire[1] + lj + '.txt', 'w') as txtfile2:
            
                    for fichier in livres:
                        try :
                            with open(csv_path+"/csvs/"+fichier+'csv', 'r') as csvfile:
                              f_csv = csv.reader(csvfile)
                              filenames = []
                              try :
                                fieldnames = next(f_csv)
                              except StopIteration as si :
                                print(si)
                              try :
                                index_i = fieldnames.index(li)
                                index_j = fieldnames.index(lj)
                                versert = fieldnames.index("livre.chapitre.verset")
                                for ligne in f_csv:
                                  x_i = ligne[index_i] 
                                  y_i =  ligne[index_j]
                                  if x_i != cell_error and y_i != cell_error :
                                      txtfile1.writelines(x_i+'\n')
                                      txtfile2.writelines(y_i+'\n')
                              except Exception as ex:
                                print(ex)
                          except Exception as ex :
                            print(ex)
                            pass

In [7]:
livres = [
    # Ancien testatement
    'GEN.', 'EXO.', 'LEV.', 'NUM.', 'DEU.', 'JOS.', 'JDG.', 'RUT.',   '1SA.', '2SA.', '1KI.', '2KI.', 
    '1CH.', '2CH.', 'EZR.', 'NEH.', 'EST.', 'JOB.', 'PSA.', 'PRO.', 'ECC.', 'SNG.',  'ISA.', 'JER.', 
    'LAM.', 'EZK.', 'DAN.', 'HOS.', 'JOL.', 'AMO.', 'OBA.', 'JON.', 'MIC.', 'NAM.', 'HAB.', 'ZEP.', 
    'HAG.', 'ZEC.', 'MAL.',

    # Nouveau testament
    'MAT.', 'MRK.', 'LUK.', 'JHN.', 'ACT.', 'ROM.', '1CO.', '2CO.', 'GAL.', 'EPH.', 'PHP.', 'COL.', 
    '1TH.', '2TH.', '1TI.', '2TI.', 'TIT.', 'PHM.', 'HEB.', 'JAS.', '1PE.',  '2PE.', '1JN.', '2JN.', 
    '3JN.', 'JUD.', 'REV.'
]

In [8]:
csv_path = "/home/XLM/data"

output_dir = "/home/jupyter/meta_XLM/XLM/data/africa"
! mkdir /home/jupyter/meta_XLM/XLM/data/africa

mkdir: cannot create directory ‘/home/jupyter/meta_XLM/XLM/data/africa’: File exists


In [50]:
data_type = "para"
langues = ["Anglais", "Bulu"]

In [51]:
get_data_from_bible(csv_path = csv_path, livres = livres, output_dir = output_dir, data_type = data_type, langues = langues)

Anglais-Bulu


In [75]:
# dossier (contenant/dans le quel stocker) les données 
%env PARA_PATH=data/africa/para
! mkdir -p $PARA_PATH

%env nCodes=30000
%env shuf_n_samples=100000

# Pourcentage des données de test et de validation (en %) 
# todo : bon ? 
%env test_size=10
%env val_size=10

env: PARA_PATH=data/africa/para
env: nCodes=30000
env: shuf_n_samples=100000
env: test_size=10
env: val_size=10


In [76]:
# tools paths
#% env TOKENIZE=$TOOLS_PATH/tokenize.sh
%env TOKENIZE=tools/tokenizer_our.sh
#% env LOWER_REMOVE_ACCENT=$TOOLS_PATH/lowercase_and_remove_accent.py
%env LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
#% env FASTBPE=$TOOLS_PATH/fastBPE/fast
%env FASTBPE=tools/fastBPE/fast

#% env OUTPATH $PROCESSED_PATH/$pair/$nCodes
# nommage du fichier en fonction de 'nCodes'
%env OUTPATH data/africa/processed/30000

! sudo mkdir -p $OUTPATH

! sudo chmod +x $FASTBPE
! sudo chmod +x get_data_hypothesis.sh
! sudo chmod +x tools/mosesdecoder/scripts/tokenizer/*.perl

env: TOKENIZE=tools/tokenizer_our.sh
env: LOWER_REMOVE_ACCENT=tools/lowercase_and_remove_accent.py
env: FASTBPE=tools/fastBPE/fast
env: OUTPATH=data/africa/processed/30000


In [77]:
%env threads_for_tokenizer=16

env: threads_for_tokenizer=16


In [78]:
! sudo chmod +x get_data_africa.sh
! ./get_data_africa.sh Anglais-Bulu 

dir data/africa/para/Anglais-Bulu already exists
dir data/africa/processed/30000/Anglais-Bulu already exists


*** Cleaning and tokenizing Anglais-Bulu data ... ***
file data/africa/para/Anglais-Bulu/Anglais-Bulu.Anglais.all already exists
file data/africa/para/Anglais-Bulu/Anglais-Bulu.Bulu.all already exists


*** split into train / valid / test ***
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error
shuf: write error: Broken pipe
shuf: write error



***build the training set for BPE tokenization (30000 codes)***


***shuf ... Generating 100000 random permutations of train data and store result in data/africa/processed/30000/Anglais-Bulu/bpe.train***
file data/africa/processed/30000/Anglais-Bulu/bpe.train already exists


***Learn the BPE vocabulary on the training set : data/africa/processed/30000/bpe.train***
file data/africa/processed/30000/Anglais-Bulu/codes already exists


***Get the

# Dependances

In [89]:
## apex
%cd /home/jupyter
! git clone https://github.com/NVIDIA/apex
%cd /home/jupyter/apex
! pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

/home/jupyter
fatal: destination path 'apex' already exists and is not an empty directory.
/home/jupyter/apex
/opt/anaconda3/lib/python3.7/site-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-orza5ue8
Created temporary directory: /tmp/pip-req-tracker-cc_q95ga
Created requirements tracker '/tmp/pip-req-tracker-cc_q95ga'
Created temporary directory: /tmp/pip-install-22y5ytw_
Processing /home/jupyter/apex
  Created temporary directory: /tmp/pip-req-build-id8hyebo
  Added file:///home/jupyter/apex to build tracker '/tmp/pip-req-tracker-cc_q95ga'
    Running setup.py (path:/tmp/pip-req-build-id8hyebo/setup.py) egg_info for package from file:///home/jupyter/apex
    Running command python setup.py egg_info
    torch.__version__  =  1.4.0
    running egg_info
    creating /tmp

In [90]:
%cd /home/jupyter/XLM

/home/jupyter/XLM


# Train

In [79]:
%env exp_id=africa
#%env batch_size=32
%env batch_size=16
%env lgs=Anglais-Bulu
# Maximum length of sentences (after BPE)
%env max_len=100

env: exp_id=africa
env: batch_size=16
env: lgs=es-fr
env: max_len=100


In [80]:
# le plus grand fichier à tensor_len phrases, on cherche le multiple de "batch_size" le plus proche de ce 
# nombre par valeur supérieur : epoch_size doit etre un multiple non nul de ce nombre (pour ne pas gaspiller) 

main_path = "/home/jupyter/XLM/data/africa/para/"
        
import io

def n_lines(file_path):
    return len(io.open(file_path, encoding='UTF-8').read().split('\n'))

tensor_len = n_lines(file_path = main_path + "Anglais-Bulu/Anglais-Bulu.Anglais.train")
print("tensor_len = " + str(tensor_len))

tensor_len = 22855


In [81]:
tensor_len=22855
#batch_size=32
batch_size=16

def getEpochSize(tensor_len, batch_size):
    i = tensor_len
    while True :
        if i%batch_size == 0 :
            return i//batch_size
        i = i + 1
        
epoch_size = getEpochSize(tensor_len, batch_size)
print("epoch_size = " + str(epoch_size))

epoch_size = 1429


In [82]:
# Et deduire epoch_size (evaluation frequency, -1 for parallel data size)
%env epoch_size=1429
#%env epoch_size=-1

env: epoch_size=1429


In [83]:
%env remove_long_sentences_train=True
%env remove_long_sentences_valid=True
%env remove_long_sentences_test=True
#--remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test

env: remove_long_sentences_train=True
env: remove_long_sentences_valid=True
env: remove_long_sentences_test=True


In [84]:
# limit the number of examples (-1 by default for non limitation)
%env train_n_samples=-1
%env valid_n_samples=-1
%env test_n_samples=-1
#--train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples

env: train_n_samples=-1
env: valid_n_samples=-1
env: test_n_samples=-1


In [85]:
# stopping criterion (if criterion does not improve 10 times)
%env stopping_criterion=_valid_mlm_ppl,10
%env eval_bleu false
%env lgs=Anglais-Bulu
%env mlm_steps=Anglais,Bulu,Anglais-Bulu

env: stopping_criterion=_valid_mlm_ppl,10
env: eval_bleu=false
env: lgs=Anglais-Bulu
env: mlm_steps=Anglais,Bulu,Anglais-Bulu


In [91]:
%env max_epoch=18
#! export NGPU=1; python -m torch.distributed.launch --nproc_per_node=$NGPU train.py --exp_name esit_mlm_tlm --dump_path ./dumped/ --data_path $OUTPATH --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --exp_id $exp_id --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples 
! python train.py --exp_name Anglais_Bulu_mlm_tlm --dump_path ./dumped/ --data_path $OUTPATH/Anglais-Bulu --lgs $lgs --clm_steps '' --mlm_steps $mlm_steps --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --batch_size $batch_size --bptt 256 --optimizer adam,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --validation_metrics _valid_mlm_ppl --stopping_criterion $stopping_criterion --eval_bleu $eval_bleu --exp_id $exp_id --remove_long_sentences_train $remove_long_sentences_train --remove_long_sentences_valid $remove_long_sentences_valid --remove_long_sentences_test $remove_long_sentences_test --train_n_samples $train_n_samples --valid_n_samples $valid_n_samples --test_n_samples $test_n_samples 

env: max_epoch=18
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm
INFO - 05/09/20 21:34:55 - 0:00:00 - ============ Initialized logger ============
INFO - 05/09/20 21:34:55 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: []
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 16
                                     beam_size: 1
                                     bptt: 256
                                     bt_src_langs: []
                                     bt_st

### Train on unsupervised MT from a pretrained model

In [93]:
#%env batch_size=...
#%env epoch_size=...

%env eval_bleu=true
# comme eval_bleu=true
%cd /home/jupyter/XLM
! chmod +x src/evaluation/multi-bleu.perl

env: eval_bleu=true
/home/jupyter/XLM


In [94]:
#%env stopping_criterion=valid_de-fr_mt_bleu,10
%env stopping_criterion=valid_Anglais-Bulu_mt_bleu,5
%env validation_metrics=valid_Anglais-Bulu_mt_bleu
%env reload_model=dumped/Anglais_Bulu_mlm_tlm/africa/best-valid_mlm_ppl.pth,dumped/Anglais_Bulu_mlm_tlm/africa/best-valid_mlm_ppl.pth
%env ae_steps=Anglais,Bulu
%env bt_steps=Anglais-Bulu-Anglais,Bulu-Anglais-Bulu

env: stopping_criterion=valid_Anglais-Bulu_mt_bleu,5
env: validation_metrics=valid_Anglais-Bulu_mt_bleu
env: reload_model=dumped/Anglais_Bulu_mlm_tlm/africa/best-valid_mlm_ppl.pth,dumped/Anglais_Bulu_mlm_tlm/africa/best-valid_mlm_ppl.pth
env: ae_steps=Anglais,Bulu
env: bt_steps=Anglais-Bulu-Anglais,Bulu-Anglais-Bulu


In [96]:
%env max_epoch=100000
%env batch_size=32
%env epoch_size=22000
#! python train.py --exp_name unsupMT_Anglais_Bulu --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH/Anglais-Bulu --lgs $lgs --ae_steps $ae_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --exp_id $exp_id        
%env mt_steps=Anglais-Bulu,Bulu-Anglais           
! python train.py --exp_name unsupMT_Anglais_Bulu --dump_path ./dumped/ --reload_model $reload_model --data_path $OUTPATH/Anglais-Bulu --lgs $lgs --ae_steps $ae_steps --mt_steps $mt_steps --bt_steps $bt_steps --word_shuffle 3 --word_dropout 0.1 --word_blank 0.1 --lambda_ae '0:1,100000:0.1,300000:0' --encoder_only false --emb_dim 1024 --n_layers 6 --n_heads 8 --dropout 0.1 --attention_dropout 0.1 --gelu_activation true --tokens_per_batch 2000 --batch_size $batch_size --bptt 256 --optimizer adam_inverse_sqrt,beta1=0.9,beta2=0.98,lr=0.0001 --epoch_size $epoch_size --max_epoch $max_epoch --eval_bleu $eval_bleu --stopping_criterion $stopping_criterion --validation_metrics $validation_metrics --exp_id $exp_id        

env: max_epoch=100000
env: batch_size=32
env: epoch_size=22000
env: mt_steps=Anglais-Bulu,Bulu-Anglais
FAISS library was not found.
FAISS not available. Switching to standard nearest neighbors search implementation.
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World size     : 1
0 - GPUs per node  : 1
0 - Master         : True
0 - Multi-node     : False
0 - Multi-GPU      : False
0 - Hostname       : african-translator-vm
INFO - 05/09/20 21:52:00 - 0:00:00 - ============ Initialized logger ============
INFO - 05/09/20 21:52:00 - 0:00:00 - accumulate_gradients: 1
                                     ae_steps: ['Anglais', 'Bulu']
                                     amp: -1
                                     asm: False
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_size: 1
                                     bptt

In [ ]:
#! rm -R dumped/unsupMT_Anglais_Bulu